#**Welcome to my Notebook**
Made by Jundan Saiful Haq for DSAI Recruitment.

In this notebook, here, i will do an Airbnb price prediction which is useful for tourists who want to find the best accommodation according to their version and their capabilities.

#1. Import some libraries, read the CSV, and merge the data
Import the common libraries first, and then read each csv files and convert into data frame. And save the test id's so that we can make submission from id's that we save